This is a notebook for HW1. Be careful that variable values persist, so you may end up getting strange errors that are only solved when you restart the runtime.

In [1]:
# These are imports and you do not need to modify these.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import numpy as np
import sklearn
import random
import math
import urllib.request


In [2]:
# Set random seed
np.random.seed(3142021)
random.seed(3142021)

def load_data():
    # Load the data
    with urllib.request.urlopen("https://www.cs.toronto.edu/~cmaddis/courses/sta314_f21/data/clean_fake.txt") as f:
        fake = [l.decode("utf-8").strip() for l in f]
    with urllib.request.urlopen("https://www.cs.toronto.edu/~cmaddis/courses/sta314_f21/data/clean_real.txt") as f:
        real = [l.decode("utf-8").strip() for l in f]
        
    # Print some sample observations
    print("Sample Real Headlines:")
    print("\n".join(real[:5]))  # Print first 5 real headlines
    print("\nSample Fake Headlines:")
    print("\n".join(fake[:5]))  # Print first 5 fake headlines


    # Each element is a string, corresponding to a headline
    data = np.array(real + fake)
    labels = np.array([0]*len(real) + [1]*len(fake))
    return data, labels

In [3]:
load_data()

Sample Real Headlines:
donald trump do you remember the year since he was elected
trump defends son over emails as moscow hits back
donald trump strategist says media wont easily give back america
anthony scaramucci who is donald trumps new comms director
donald trumps mobile phone use worries security experts

Sample Fake Headlines:
trump warns of vote flipping on machines
this election is not about trump its about a giant middle finger to washington dc
more on trump populism and how it can be controlled by government
trump bollywood ad meant to sway indian american voters is an hilarious fail
dems could be up on charges for inciting trump rally violence


(array(['donald trump do you remember the year since he was elected',
        'trump defends son over emails as moscow hits back',
        'donald trump strategist says media wont easily give back america',
        ...,
        'breaking donald trump makes major clinton indictment announcement details',
        'no hate crimes have not intensified since trump election',
        'who to blame for president trump'], dtype='<U198'),
 array([0, 0, 0, ..., 1, 1, 1]))

In [4]:
def process_data(data, labels):
    """
    Preprocess a dataset of strings into vector representations.

    Parameters
    ----------
    data: numpy array
        An array of N strings.
    labels: numpy array
        An array of N integer labels.

    Returns
    -------
    train_X: numpy array
        Array with shape (N, D) of N inputs.
    train_Y:
        Array with shape (N,) of N labels.
    val_X:
        Array with shape (M, D) of M inputs.
    val_Y:
        Array with shape (M,) of M labels.
    test_X:
        Array with shape (M, D) of M inputs.
    test_Y:
        Array with shape (M,) of M labels.
    """

   # Step 1: Split the dataset into training, validation, and test sets (70/15/15)
    train_X, temp_X, train_Y, temp_Y = train_test_split(
        data, labels, test_size=0.30, stratify=labels, random_state=3142021
    )
    val_X, test_X, val_Y, test_Y = train_test_split(
        temp_X, temp_Y, test_size=0.50, stratify=temp_Y, random_state=3142021
    )

    # Step 2: Preprocess the data using CountVectorizer
    vectorizer = CountVectorizer()

    # Fit the vectorizer only on the training data, then transform all sets
    train_X = vectorizer.fit_transform(train_X)
    val_X = vectorizer.transform(val_X)
    test_X = vectorizer.transform(test_X)

    # Convert sparse matrices to dense arrays for easier use
    train_X = train_X.toarray()
    val_X = val_X.toarray()
    test_X = test_X.toarray()

    # Return the training, validation, and test sets
    return train_X, train_Y, val_X, val_Y, test_X, test_Y


In [5]:
def select_knn_model(train_X, val_X, train_Y, val_Y):
    """
    Test k in {1, ..., 20} and return the a k-NN model
    fitted to the training set with the best validation loss.

    Parameters
    ----------
    train_X: numpy array
        Array with shape (N, D) of N inputs.
    train_Y: numpy array
        Array with shape (M, D) of M inputs.
    train_Y: numpy array
        Array with shape (N,) of N labels.
    val_Y: numpy array
        Array with shape (M,) of M labels.

    Returns
    -------
    best_model : KNeighborsClassifier
        The best k-NN classifier fit on the training data 
    and selected according to validation loss.
    best_k : int
        The best k value according to validation loss.
    """
    best_k = None
    best_model = None
    best_accuracy = 0

    # Test k values from 1 to 20
    for k in range(1, 21):
        # Initialize the k-NN classifier with the current k
        model = KNeighborsClassifier(n_neighbors=k)
        
        # Train the model on the training data
        model.fit(train_X, train_Y)
        
        # Make predictions on the validation data
        val_pred = model.predict(val_X)
        
        # Calculate accuracy on the validation set
        accuracy = accuracy_score(val_Y, val_pred)
        
        # Update the best model if this one performs better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_k = k
            best_model = model

    return best_model, best_k

Run the next cell to get the output.

In [6]:
def main():
    data, labels = load_data()
    train_X, train_Y, val_X, val_Y, test_X, test_Y = process_data(data, labels)

    best_model, best_k = select_knn_model(train_X, val_X, train_Y, val_Y)
    test_accuracy = best_model.score(test_X, test_Y)
    print("Selected K: {}".format(best_k))
    print("Test Acc: {}".format(test_accuracy))

In [7]:
main()

Sample Real Headlines:
donald trump do you remember the year since he was elected
trump defends son over emails as moscow hits back
donald trump strategist says media wont easily give back america
anthony scaramucci who is donald trumps new comms director
donald trumps mobile phone use worries security experts

Sample Fake Headlines:
trump warns of vote flipping on machines
this election is not about trump its about a giant middle finger to washington dc
more on trump populism and how it can be controlled by government
trump bollywood ad meant to sway indian american voters is an hilarious fail
dems could be up on charges for inciting trump rally violence


NameError: name 'accuracy_score' is not defined